# <a id='toc1_'></a>[Bring Your Own PyTorch Container to Amazon SageMaker Pipeline](#toc0_)

---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook.

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-2/advanced_functionality|byoc-pytorch-container|byoc_pytorch_container.ipynb)

---

With Amazon SageMaker, you can package your own deep learning algorithms into PyTorch container and train it in the SageMaker. This notebook will guide you through an example that show you how to build a PyTorch container for SageMaker and use it for training, real-time inference and batch transform. 

1. Before run this notebook, we need to build a training container and an inference container.

2. After we build the PyTorch containers, we can build an ML pipeline include:
    * Preprocessing the abalone dataset
    * Training a PyTorch model with own container
    * Register the PyTorch model to SageMaker Model Registry

3. When we finish the ML Pipe, we can:
    * Check and update the model status
    * Deploy the model to Amazon SageMaker Endpoint from Model Registry
    * Do real-time inference on Amazon SageMaker Endpoint
    * Do batch inference by Amazon SageMaker Transform



**Table of contents**<a id='toc0_'></a>    
- [Bring Your Own PyTorch Container to Amazon SageMaker Pipeline](#toc1_)    
  - [Build PyTorch container](#toc1_1_)    
  - [Build ML Pipeline](#toc1_2_)    
    - [Data Preparation Step](#toc1_2_1_)    
    - [PyTorch Model Training Step](#toc1_2_2_)    
    - [Register model](#toc1_2_3_)    
    - [Define Pipeline](#toc1_2_4_)    
    - [Execute the Pipeline](#toc1_2_5_)    
    - [Save JSON file](#toc1_2_6_)    
  - [Inference](#toc1_3_)    
    - [Real-time Inference](#toc1_3_1_)    
      - [Update Model Status](#toc1_3_1_1_)    
      - [Deploy model to SageMaker Endpoint](#toc1_3_1_2_)    
      - [Real-time Inference on SageMaker Endpoint](#toc1_3_1_3_)    
    - [Batch Transform](#toc1_3_2_)    
  - [Cleaning up resources](#toc1_4_)    
  - [Notebook CI Test Results](#toc1_5_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_1_'></a>[Build PyTorch container](#toc0_)

You can build training and inference docker by run bash command on terminal or run bash command in Jupyter notebook.

1. Build training container from terminal

    bash
    ```
    cd docker/train
    ./build_train_docker.sh
    ```

2. Build inference container from terminal
    bash
    ```
    cd docker/inference
    ./build_inference_docker.sh
    ```

For PyTorch container, we need to use difference base container to build the image


In [ ]:
import sys

! pip install --upgrade sagemaker

In [ ]:
import os
import boto3
import sagemaker
from sagemaker.processing import (
    ProcessingOutput,
)

from sagemaker import Model
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.steps import ProcessingStep


from sagemaker.workflow.functions import Join
from sagemaker.workflow.execution_variables import ExecutionVariables
from sagemaker.workflow.pipeline_context import PipelineSession

In [ ]:
# Create the SageMaker Session

region = sagemaker.Session().boto_region_name
sm_client = boto3.client("sagemaker")
boto_session = boto3.Session(region_name=region)
sagemaker_session = sagemaker.session.Session(boto_session=boto_session, sagemaker_client=sm_client)
account_id = boto3.client("sts").get_caller_identity().get("Account")

# Create a Pipeline Session
pipeline_session = PipelineSession()

In [ ]:
# Define variables and parameters needed for the Pipeline steps
role = sagemaker.get_execution_role()
pipeline_name = "DEMO-pipeline-PyTorch"
endpoint_name = "pipeline-demo-endpoint"

default_bucket = sagemaker_session.default_bucket()
base_job_prefix = "abalone-pipeline-deploy"

processing_instance_count = ParameterInteger(name="ProcessingInstanceCount", default_value=1)
training_instance_type = ParameterString(
    name="TrainingInstanceType", default_value="ml.g4dn.xlarge"
)
input_data = ParameterString(
    name="InputDataUrl",
    default_value=f"s3://agemaker-example-files-prod-us-east-1/datasets/tabular/uci_abalone/abalone.csv",
)

## <a id='toc1_2_'></a>[Build ML Pipeline](#toc0_)

### <a id='toc1_2_1_'></a>[Data Preparation Step](#toc0_)

An SKLearn processor is used to prepare the dataset for the Training Step. Using the script `preprocess.py`, the dataset is featurized and split into train, test, and validation datasets.

The output of this step is used as the input to the TuningStep

In [ ]:
!mkdir -p code

In [ ]:
%%writefile code/preprocess.py

"""Feature engineers the abalone dataset."""
import argparse
import logging
import os
import pathlib
import requests
import tempfile

import boto3
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler())


# Since we get a headerless CSV file we specify the column names here.
feature_columns_names = [
    "sex",
    "length",
    "diameter",
    "height",
    "whole_weight",
    "shucked_weight",
    "viscera_weight",
    "shell_weight",
]
label_column = "rings"

feature_columns_dtype = {
    "sex": str,
    "length": np.float64,
    "diameter": np.float64,
    "height": np.float64,
    "whole_weight": np.float64,
    "shucked_weight": np.float64,
    "viscera_weight": np.float64,
    "shell_weight": np.float64,
}
label_column_dtype = {"rings": np.float64}


def merge_two_dicts(x, y):
    """Merges two dicts, returning a new copy."""
    z = x.copy()
    z.update(y)
    return z


if __name__ == "__main__":
    logger.debug("Starting preprocessing.")
    parser = argparse.ArgumentParser()
    parser.add_argument("--input-data", type=str, required=True)
    args = parser.parse_args()

    base_dir = "/opt/ml/processing"
    pathlib.Path(f"{base_dir}/data").mkdir(parents=True, exist_ok=True)
    input_data = args.input_data
    bucket = input_data.split("/")[2]
    key = "/".join(input_data.split("/")[3:])

    logger.info("Downloading data from bucket: %s, key: %s", bucket, key)
    fn = f"{base_dir}/data/abalone-dataset.csv"
    s3 = boto3.resource("s3")
    s3.Bucket(bucket).download_file(key, fn)

    logger.debug("Reading downloaded data.")
    df = pd.read_csv(
        fn,
        header=None,
        names=feature_columns_names + [label_column],
        dtype=merge_two_dicts(feature_columns_dtype, label_column_dtype),
    )
    os.unlink(fn)

    logger.debug("Defining transformers.")
    numeric_features = list(feature_columns_names)
    numeric_features.remove("sex")
    numeric_transformer = Pipeline(
        steps=[
            ("imputer", SimpleImputer(strategy="median")),
            ("scaler", StandardScaler()),
        ]
    )

    categorical_features = ["sex"]
    categorical_transformer = Pipeline(
        steps=[
            ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
            ("onehot", OneHotEncoder(handle_unknown="ignore")),
        ]
    )

    preprocess = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, numeric_features),
            ("cat", categorical_transformer, categorical_features),
        ]
    )

    logger.info("Applying transforms.")
    y = df.pop("rings")
    X_pre = preprocess.fit_transform(df)
    y_pre = y.to_numpy().reshape(len(y), 1)

    X = np.concatenate((y_pre, X_pre), axis=1)

    logger.info("Splitting %d rows of data into train, validation, test datasets.", len(X))
    np.random.shuffle(X)
    train, validation, test = np.split(X, [int(0.7 * len(X)), int(0.85 * len(X))])

    logger.info("Writing out datasets to %s.", base_dir)
    pd.DataFrame(train).to_csv(f"{base_dir}/train/train.csv", header=False, index=False)
    pd.DataFrame(validation).to_csv(
        f"{base_dir}/validation/validation.csv", header=False, index=False
    )
    pd.DataFrame(test).to_csv(f"{base_dir}/test/test.csv", header=False, index=False)
    pd.DataFrame(test).to_csv(f"{base_dir}/batch/test.csv", header=False, index=False)

In [ ]:
# Process the training data step using a python script.
# Split the training data set into train, test, and validation datasets
# When defining the ProcessingOutput destination as a dynamic value using the
# Pipeline Execution ID, caching will not be in effect as each time the step runs,
# the step definition changes resulting in new execution. If caching is required,
# the ProcessingOutput definition should be status

sklearn_processor = SKLearnProcessor(
    framework_version="0.23-1",
    instance_type="ml.m5.xlarge",
    instance_count=processing_instance_count,
    base_job_name=f"{base_job_prefix}/sklearn-abalone-preprocess",
    sagemaker_session=pipeline_session,
    role=role,
)

processor_run_args = sklearn_processor.run(
    outputs=[
        ProcessingOutput(
            output_name="train",
            source="/opt/ml/processing/train",
            destination=Join(
                on="/",
                values=[
                    "s3:/",
                    default_bucket,
                    base_job_prefix,
                    ExecutionVariables.PIPELINE_EXECUTION_ID,
                    "PreprocessAbaloneDataForTrain",
                ],
            ),
        ),
        ProcessingOutput(
            output_name="validation",
            source="/opt/ml/processing/validation",
            destination=Join(
                on="/",
                values=[
                    "s3:/",
                    default_bucket,
                    base_job_prefix,
                    ExecutionVariables.PIPELINE_EXECUTION_ID,
                    "PreprocessAbaloneDataForTrain",
                ],
            ),
        ),
        ProcessingOutput(
            output_name="test",
            source="/opt/ml/processing/test",
            destination=Join(
                on="/",
                values=[
                    "s3:/",
                    default_bucket,
                    base_job_prefix,
                    ExecutionVariables.PIPELINE_EXECUTION_ID,
                    "PreprocessAbaloneDataForTrain",
                ],
            ),
        ),
        ProcessingOutput(
            output_name="batch",
            source="/opt/ml/processing/batch",
            destination=Join(
                on="/",
                values=["s3:/", default_bucket, base_job_prefix, "inference-test-input"],
            ),
        ),
    ],
    code="code/preprocess.py",
    arguments=["--input-data", input_data],
)

step_process = ProcessingStep(
    name="preprocess-abalone-data-for-train",
    step_args=processor_run_args,
)

### <a id='toc1_2_2_'></a>[PyTorch Model Training Step](#toc0_)

Create PyTorch training docker

In [ ]:
%%sh
# The name of our algorithm
algorithm_name=torch_sample_train

cd docker/train

chmod +x code/train

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration 
region=$(aws configure get region)
fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
# $(aws ecr get-login --region ${region} --no-include-email)
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

aws ecr get-login-password --region ${region} | docker login --username AWS --password-stdin 763104351884.dkr.ecr.${region}.amazonaws.com


# Get the login command from ECR in order to pull down the SageMaker PyTorch image
# Build the docker image locally with the image name and then push it to ECR with the full name.
docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}
docker push ${fullname}


Create PyTorch inference docker

In [ ]:
%%sh
# The name of our algorithm
algorithm_name=torch_sample_inference

cd docker/inference

chmod +x code/serve.py

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration 
region=$(aws configure get region)
fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

aws ecr get-login-password --region ${region} | docker login --username AWS --password-stdin 763104351884.dkr.ecr.${region}.amazonaws.com


# Get the login command from ECR in order to pull down the SageMaker PyTorch image
# Build the docker image locally with the image name and then push it to ECR with the full name.
docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}
docker push ${fullname}


create training estimator

In [ ]:
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput

instance_type = ParameterString(name="TrainingInstanceType", default_value="ml.g4dn.xlarge")

model_path = f"s3://{default_bucket}/AbaloneTrain"

image_uri_train = f"{account_id}.dkr.ecr.{region}.amazonaws.com/torch_sample_train:latest"
image_uri_inference = f"{account_id}.dkr.ecr.{region}.amazonaws.com/torch_sample_inference:latest"

pt_train = Estimator(
    image_uri=image_uri_train,
    instance_type=instance_type,
    instance_count=1,
    output_path=model_path,
    role=role,
    sagemaker_session=pipeline_session,
)

In [ ]:
train_args = pt_train.fit(
    inputs={
        "train": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs["train"].S3Output.S3Uri,
            content_type="text/csv",
        ),
        "validation": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                "validation"
            ].S3Output.S3Uri,
            content_type="text/csv",
        ),
    }
)

In [ ]:
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep


step_train = TrainingStep(
    name="Train",
    step_args=train_args,
)

### <a id='toc1_2_3_'></a>[Register model](#toc0_)

In [ ]:
from sagemaker.workflow.step_collections import RegisterModel

mpg_name = "pipeline-demo-mpg"


model_approval_status = "PendingManualApproval"
register_step = RegisterModel(
    name="abalone-demo",
    estimator=pt_train,
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.g4dn.xlarge"],
    transform_instances=["ml.g4dn.xlarge"],
    model_package_group_name=mpg_name,
    approval_status=model_approval_status,
    image_uri=image_uri_inference,
)

### <a id='toc1_2_4_'></a>[Define Pipeline](#toc0_)

In [ ]:
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_count,
        training_instance_type,
        input_data,
    ],
    steps=[
        step_process,
        step_train,
        register_step,
    ],
    sagemaker_session=pipeline_session,
)

### <a id='toc1_2_5_'></a>[Execute the Pipeline](#toc0_)

In [ ]:
pipeline.upsert(role_arn=role)

In [ ]:
pipeline.start()

### <a id='toc1_2_6_'></a>[Save JSON file](#toc0_)


In [ ]:
import json

definition = json.loads(pipeline.definition())
with open("./pipeline-definition.json", "w") as outfile:
    outfile.write(pipeline.definition())

## <a id='toc1_3_'></a>[Inference](#toc0_)

### <a id='toc1_3_1_'></a>[Real-time Inference](#toc0_)

check the model package group information

In [ ]:
import boto3

# Create a SageMaker client
sm_client = boto3.client("sagemaker")

mpg_Name = "pipeline-demo-mpg"
sm_client.list_model_packages(ModelPackageGroupName=mpg_Name)

#### <a id='toc1_3_1_1_'></a>[Update Model Status](#toc0_)

In [ ]:
# update model status
model_package_summary = sm_client.list_model_packages(
    ModelPackageGroupName=mpg_Name, SortBy="CreationTime", SortOrder="Descending"
)

model_package_arn = model_package_summary["ModelPackageSummaryList"][0]["ModelPackageArn"]
model_package_update_input_dict = {
    "ModelPackageArn": model_package_arn,
    "ModelApprovalStatus": "Approved",
}
model_package_update_response = sm_client.update_model_package(**model_package_update_input_dict)

In [ ]:
model_package_summarys

#### <a id='toc1_3_1_2_'></a>[Deploy model to SageMaker Endpoint](#toc0_)

In [ ]:
mpg_info = sm_client.describe_model_package(ModelPackageName=model_package_arn)
mpg_info["InferenceSpecification"]["Containers"][0]["ModelDataUrl"]

# deploy the Pytorch Model to Amazon SageMaker Endpoint
from sagemaker import ModelPackage

model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)
model.deploy(endpoint_name=endpoint_name, initial_instance_count=1, instance_type="ml.g4dn.xlarge")

#### <a id='toc1_3_1_3_'></a>[Real-time Inference on SageMaker Endpoint](#toc0_)

In [ ]:
import boto3

s3 = boto3.client("s3")
# Input the test.csv oject name. Need to modify with your pipeline ID
object_name = f"{base_job_prefix}/inference-test-input/test.csv"
test_csv_s3_path = f"s3://{default_bucket}/{object_name}"
s3.download_file(default_bucket, object_name, "test.csv")

In [ ]:
import boto3
import pandas as pd
import io

csv_buffer = open("test.csv")
my_payload_as_csv = csv_buffer.read()
client = boto3.client("sagemaker-runtime")
response = client.invoke_endpoint(
    EndpointName=endpoint_name, Body=my_payload_as_csv, ContentType="text/csv"
)

output = response["Body"].read()
print(output)

### <a id='toc1_3_2_'></a>[Batch Transform](#toc0_)

Get model name

In [ ]:
models = sm_client.search(
    Resource="Model",
    SearchExpression={
        "Filters": [
            {
                "Name": "Model.Containers.ModelPackageName",
                "Operator": "Equals",
                "Value": model_package_arn,
            },
        ]
    },
)["Results"]

model_name = models[0]["Model"]["Model"]["ModelName"]

In [ ]:
from sagemaker.transformer import Transformer

# Create a Pipeline Session
instance_type = "ml.g4dn.xlarge"

batch_transform_output_path = f"s3://{default_bucket}/batch_result"
transformer = Transformer(
    model_name=model_name,
    instance_count=1,
    instance_type=instance_type,
    assemble_with="Line",
    accept="text/csv",
    output_path=batch_transform_output_path,
)


transformer.transform(test_csv_s3_path, content_type="text/csv")

## <a id='toc1_4_'></a>[Cleaning up resources](#toc0_)

Users are responsible for cleaning up resources created when running this notebook. Specify the ModelName, ModelPackageName, and ModelPackageGroupName that need to be deleted. The model names are generated by the CreateModel step of the Pipeline and the property values are available only in the Pipeline context. To delete the models created by this pipeline, navigate to the Model Registry and Console to find the models to delete.

In [ ]:
# Create a SageMaker client
sm_client = boto3.client("sagemaker")

# Delete SageMaker Models
sm_client.delete_model(ModelName=model_name)

# # Delete Model Packages
model_package_list = model_package_summary["ModelPackageSummaryList"]
for i in range(len(model_package_list)):
    tmp_arn = model_package_list[i]["ModelPackageArn"]
    sm_client.delete_model_package(ModelPackageName=tmp_arn)

# Delete the Model Package Group
sm_client.delete_model_package_group(ModelPackageGroupName=mpg_Name)

# Delete the Pipeline
sm_client.delete_pipeline(PipelineName=pipeline_name)

# Delete endpoint
sm_client.delete_endpoint(EndpointName=endpoint_name)

# Delete endpoint configuration
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_name)

## <a id='toc1_5_'></a>[Notebook CI Test Results](#toc0_)

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.


![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-1/advanced_functionality|byoc-pytorch-container|byoc_pytorch_container.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-2/advanced_functionality|byoc-pytorch-container|byoc_pytorch_container.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-1/advanced_functionality|byoc-pytorch-container|byoc_pytorch_container.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ca-central-1/advanced_functionality|byoc-pytorch-container|byoc_pytorch_container.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/sa-east-1/advanced_functionality|byoc-pytorch-container|byoc_pytorch_container.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-1/advanced_functionality|byoc-pytorch-container|byoc_pytorch_container.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-2/advanced_functionality|byoc-pytorch-container|byoc_pytorch_container.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-3/advanced_functionality|byoc-pytorch-container|byoc_pytorch_container.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-central-1/advanced_functionality|byoc-pytorch-container|byoc_pytorch_container.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-north-1/advanced_functionality|byoc-pytorch-container|byoc_pytorch_container.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-1/advanced_functionality|byoc-pytorch-container|byoc_pytorch_container.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-2/advanced_functionality|byoc-pytorch-container|byoc_pytorch_container.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-1/advanced_functionality|byoc-pytorch-container|byoc_pytorch_container.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-2/advanced_functionality|byoc-pytorch-container|byoc_pytorch_container.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-south-1/advanced_functionality|byoc-pytorch-container|byoc_pytorch_container.ipynb)
